In [ ]:
from pathlib import Path
from bressen import BasisGegevens, read_files
from bressen.kering import get_kering_geometry
from bressen.styles import add_styles_to_geopackage
from bressen.geometries import get_closest_feature, project_point, get_containing_feature, get_offsets
import geopandas as gpd
from shapely.geometry import LineString, Point

## Paden naar gegevens
Gegevens zijn aangeleverd in verschillende directories:
- 01_AGV_Dijktrajecten_Legger: dijktrajecten van waterschap Amstel Gooi en Vecht
- 02_ARK_Kering: kering van het Amsterdam-Rijnkanaal
- 03_Doorbraaklocaties: de locaties van de doorbraken
- 04_Watervlakken: alle beschikbare watervlakken

BASISGEGEVENS_GPKG en BRESSEN_GPKG verwijzen naar de basisgegevens (keringen, peil- en watervlakken) en het bressen-bestand

In [ ]:
DATA_DIR = Path(r"d:\projecten\D2405.waternet.bressen\01.gegevens")
BRESSEN_SHP = DATA_DIR.joinpath("03_Doorbraaklocaties", "Bres_Test_060324.shp")

BASISGEGEVENS_GPKG = DATA_DIR / "basisgegevens.gpkg"
BRESSEN_GPKG = DATA_DIR / "bressen.gpkg"

## Inlezen
Inlezen van basisgegevens en bressen

In [ ]:
# %% Inlezen basisgegevens vanuit GPKG
basis_gegevens = BasisGegevens.from_gpkg(BASISGEGEVENS_GPKG)

# Inlezen bressen vanuit meerdere bestanden
bressen_gdf = read_files([
    DATA_DIR.joinpath("03_Doorbraaklocaties", "240122_breslocaties_gelijk_groter_1000.shp"),
    DATA_DIR.joinpath("03_Doorbraaklocaties", "240212_breslocaties_kleiner_1000_angle.shp"),
    DATA_DIR.joinpath("03_Doorbraaklocaties", "240212_breslocaties_Waterkeringen_Blaeu_ARK.shp"),
    ])

# Filteren lege geometrieen
bressen_gdf = bressen_gdf[~bressen_gdf.geometry.isna()]

# Filteren namen die we (voor nu) niet kunnen processen om verschillende redenen
ignore_kwknaam = ["Kanteldijk N201", "Geniedijk - Legmeerdijk"]
bressen_gdf = bressen_gdf[~bressen_gdf.DWKNAAM.isin(ignore_kwknaam)]

# Resetten index tot een fid
bressen_gdf.reset_index(inplace=True)
bressen_gdf.index += 1

# Bressen wegschrijven naar GPKG
bressen_gdf.to_file(BRESSEN_GPKG, layer="bressen", engine="pyogrio")

## Verwerken Bressen
We hanteren de volgende instellingen
`x1`: offset van de bres t.o.v. kering in meter
`x2`: breedte van de bres haaks op de kering in meter
`x3`: lengte van de bres langs de kering in meter

In [ ]:
# instellingen
tolerance = 0.1
x1 = 50
x2 = 40
x3 = 200

# init data-lijst (wordt later GeoDataFrame)
data = []

# itereren over bressen
for row in bressen_gdf.itertuples():

    # 1. selecteer dichtsbijzijnde kering binnen tolerantie
    kering_geometry = get_kering_geometry(
        row,
        keringen=basis_gegevens.keringen,
        min_length=x3,
        max_distance=tolerance,
        max_line_extends=1,
        )
        
    # 2. bepaal offset_locatie (1) naast watervlak (2) in laagste peilgebied (3) binnen beheergebied

    # haal watervlak op
    watervlak = get_closest_feature(row, basis_gegevens.watervlakken, max_distance=x1)
    if watervlak is not None: # (1) naast watervlak
        offset_distance = max(min(x1, watervlak.geometry.distance(row.geometry)), tolerance)
        offsets = get_offsets(kering_geometry, offset_distance)
        offset_points = offsets.apply(lambda x: project_point(x, row.geometry))
        idx = offset_points.distance(watervlak.geometry).sort_values(ascending=False).index[0]
        offset_locatie = "naast watervlak"
        if offset_distance != x1:
            offsets = get_offsets(kering_geometry, x1, check_emtpy_lines=False)
    else:
        # zoek 1 of meerdere peilvlakken
        offsets = get_offsets(kering_geometry, x1)
        offset_points = (project_point(line, row.geometry) for line in offsets)
        peilvlakken = [get_containing_feature(geometry, basis_gegevens.peilvlakken) for geometry in offset_points]
        peilen = [i.peil if i is not None else None for i in peilvlakken]
        if all(peilen): # (2) tussen twee peilvlakken, we kiezen laagste peil
            if peilen[0] == peilen[1]:
                print(f"Peilvlak(ken) aan beiden zijden van bres met fid {row.Index} hebben zelfde peil: {peilen[0]} m NAP op afstand {x1}")
                continue
                raise ValueError(f"Peilvlak(ken) aan beiden zijden van bres met fid {row.Index} hebben zelfde peil: {peilen[0]} m NAP op afstand {x1}")
            else:
                idx = peilen.index(min(peilen))
                offset_locatie = "laagste peilvlak"
        elif not any(peilen):
            print(f"Geen peilvlakken aan een zijde van bres met fid {row.Index} afstand {x1}")
            continue
            raise ValueError(f"Geen peilvlakken aan een zijde van bres met fid {row.Index} afstand {x1}")
        else: # (3) we kiezen het peilvlak binnen het beheergebied
            idx = next((idx for idx, i in enumerate(peilen) if i is not None))
            offset_locatie = "binnen beheergebied"

    bres_offset = offsets[idx]  

    # 3. bres_offset vertalen naar bres_poly
    mid_distance = bres_offset.project(row.geometry)
    min_distance= max(mid_distance - x3/2, 0)
    max_distance= min(mid_distance + x3/2, bres_offset.length)
    line_start = bres_offset.interpolate(min_distance)
    line_end = bres_offset.interpolate(max_distance)
    distances = (bres_offset.project(Point(i)) for i in bres_offset.coords)
    line_vertices = [bres_offset.interpolate(i) for i in distances if (i > min_distance) and (i < max_distance)]
    bres_line = LineString(([line_start]+ line_vertices + [line_end]))
    bres_poly = bres_line.buffer(x2/2, cap_style="flat")

    # 4. toevoegen aan data
    row_dict = row._asdict()
    row_dict["geometry"] = bres_poly
    row_dict["offset_locatie"] = offset_locatie
    data += [row_dict]


## Wegschrijven resultaat

In [ ]:
# Wegschrijven bresvlakken
bresvlakken_gdf = gpd.GeoDataFrame(data, crs=bressen_gdf.crs)
bresvlakken_gdf.drop(columns="Index", inplace=True)
bresvlakken_gdf.to_file(BRESSEN_GPKG, layer="bresvlakken", engine="pyogrio")

# toevoegen styles aan GeoPackage
add_styles_to_geopackage(BRESSEN_GPKG)